$\mathrm{\huge{Neolithic\,Circular\,Enclosures}}$

$\mathrm{\large{Notebook\,II:\, Calculation\,and\,visualization\,of\,sun\,and\,moon\,paths\,on\,the\,horizon}}$


$\mathrm{Gerd\,Graßhoff^{1,2,3}, Gordon\,Fischer^{2}}$
  
1: Humboldt Universität zu Berlin, 2: Excellence Cluster Topoi  
3: Max Planck Institute for history  of science  
Corresponding mail: gerd.grasshoff@hu-berlin.de, gordon.fischer@topoi.org  
Date: 18.05.2017

**goal of research:** Calculation of sun, moon & star positions (altitude, azimuth) on the horizon
        
**keywords**: Neolithic Circular Enclosures, star position, altitude, azimuth

**Citable Class and python packages**

In [3]:
#IMPORT LIBRARIES
from citableclass import Citable 
import csv
import json
#import astronomia
import ephem
from ephem import *
import numpy as np
from ephem import stars 
import pandas as pd
db=stars.db
import numpy as np
import math
import sys
from IPython.display import display
from ipywidgets import widgets, interactive
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from collections import Counter
from ephem import Equatorial, Ecliptic
from IPython.display import Image, clear_output
from ipywidgets import FloatSlider,IntSlider,Dropdown,Label, Button, Layout, Textarea, HBox, VBox, ToggleButtons, RadioButtons, jslink,interact, interact_manual,Output, Text, FloatText, Select

**Helper functions**

In [4]:
# HELPER FUNCTIONS
#STAR CATALOGUE
pyephembsc = []
cit=Citable("BAAP00028", formats="et")
reader = cit.csv()
for row in reader:
    pyephembsc.append(row)
stars=[]
for j in range(len(pyephembsc)):
    stars.append(pyephembsc[j][0])
starsforwidget=stars;

def CheckStars(starname):
    try:
        selectedstar=ephem.star(starname)
        return starname
    except:
         #print("no star")
        pass    
correctedStarlist=[]
for j in range(len(starsforwidget)):
    v1 = CheckStars(starsforwidget[j])
    if v1 != "no star":
        correctedStarlist.append(v1)
starsforwidget = [x for x in correctedStarlist if x is not None]

# Analysis function
calculation of sun and moon path on horizon for  
   - epoch
   - region (object)
   

In [5]:
# FUNCTION FOR SUN, MOON & STARS
def AzimuthAltitudeFull(date, starname1,starname2, starname3, Anlage, special, xmin, xmax, ymin, ymax, pressureval, zoom):

    gatech = ephem.Observer()
    gatech.pressure=pressureval
    v=Citable(Anlage, formats="et")
    anlage=v.json()
    # Korrektur der Werte
    profil=[]
    profilAz=[]
    profilAlt=[]
    anlage=anlage["general"]
    az = anlage["profile"]["azimuth"]
    zenitwinkel= anlage["profile"]["altitude"]

#LOOP  
    for j in range(len(az)):
   
        if  float(az[j])>=90 and float(az[j])<360:
            profil.append(((-1*float(az[j]))+360+90,90-float(zenitwinkel[j]) ))
            
        if float(az[j])<90:
            profil.append(((-1*float(az[j]))+90,90-float(zenitwinkel[j]) ))
            

    profil = sorted(profil,key=lambda l:l[0], reverse=False)
    profilAz=[row[0] for row in profil]
    profilAlt=[row[1] for row in profil]
    gatech.lat, gatech.lon =str(anlage["coordinates"]["latitude"]) , str(anlage["coordinates"]["longitude"])
    azimuthStar1=[]
    altitudeStar1=[]
    azimuthStar2=[]
    altitudeStar2=[]
    azimuthStar3=[]
    altitudeStar3=[]
    azimuthMoon=[]
    altitudeMoon=[]
    azimuthSun=[]
    altitudeSun=[]
    
    p = Equatorial('90', '0', epoch=str(date.split("/")[0]))
    eclipt= float(Ecliptic(p).lat)*180/3.14
   
    
    if special != "no selection":
        date= ephem.date((int(date.split("/")[0]),1,1))
        orbitPoints= ['vernal_equinox','summer_solstice','autumnal_equinox','winter_solstice']
        dates= {}
        dates['vernal_equinox_start']= ephem.next_vernal_equinox(date)
        
        if special=="vernal_equinox":
            date= ephem.next_vernal_equinox(date)
        if special=="summer_solstice":
            date= ephem.next_summer_solstice(dates['vernal_equinox_start'])
        if special=="autumnal_equinox":
            date= ephem.next_autumnal_equinox(dates['vernal_equinox_start'])
        if special=="winter_solstice":
            date = ephem.next_winter_solstice(dates['vernal_equinox_start'])
        if special =="all special events":
            dateFP = ephem.next_vernal_equinox(date)
            dateFP= str(dateFP).split(" ")[0]
            dateSSW = ephem.next_summer_solstice(dates['vernal_equinox_start'])
            dateSSW= str(dateSSW).split(" ")[0]
            dateHP = ephem.next_autumnal_equinox(dates['vernal_equinox_start'])
            dateHP= str(dateHP).split(" ")[0]
            dateWSW =ephem.next_winter_solstice(dates['vernal_equinox_start'])
            dateWSW= str(dateWSW).split(" ")[0]
           
       
        date=str(date).split(" ")[0]
    else: 
        date=date
    
    templist1=[]
    templist2=[]
    templist3=[]
    for j in range(96):
        
        gatech.date =str(date) +' '+str(j/4.)+":00:00"
        #Star
        if starname1 != "no star":
            try: 
                ephem.star(starname1)
                selectedstar=ephem.star(starname1)
                selectedstar.compute(gatech)
                mag1=selectedstar.mag
                templist1.append((float(selectedstar.az*180/3.14), float(selectedstar.alt*180/3.14) ))
                templist1 = sorted(templist1,key=lambda l:l[0], reverse=False)
                azimuthStar1 = [row[0] for row in templist1]
                altitudeStar1 = [row[1] for row in templist1]
               

            except:
                if j ==1:
                    print("Star not available on ephem library")
                pass
     
        if starname2 != "no star":
            try: 
                ephem.star(starname2)
                selectedstar=ephem.star(starname2)
                selectedstar.compute(gatech)
                mag2=selectedstar.mag
                templist2.append((float(selectedstar.az*180/3.14), float(selectedstar.alt*180/3.14) ))
                templist2 = sorted(templist2,key=lambda l:l[0], reverse=False)
                azimuthStar2 = [row[0] for row in templist2]
                altitudeStar2 = [row[1] for row in templist2]
            
            except:
                if j ==1:
                    print("Star not available on ephem library")
                pass
            
        if starname3 != "no star":
            try: 
                ephem.star(starname3)
                selectedstar=ephem.star(starname3)
                selectedstar.compute(gatech)
                mag3=selectedstar.mag
                templist3.append((float(selectedstar.az*180/3.14), float(selectedstar.alt*180/3.14) ))
                templist3 = sorted(templist3,key=lambda l:l[0], reverse=False)
                azimuthStar3 = [row[0] for row in templist3]
                altitudeStar3 = [row[1] for row in templist3]
             

            except:
                if j ==1:
                    print("Star not available on ephem library")
                pass
        #Moon
        moon=ephem.Moon()
        moon.compute(gatech)
        azimuthMoon.append(float(moon.az*180/3.14))
        altitudeMoon.append(float(moon.alt*180/3.14))
        #Sun
        sun=ephem.Sun()
        sun.compute(gatech)
        azimuthSun.append(float(sun.az*180/3.14))
        altitudeSun.append(float(sun.alt*180/3.14))
    
    azimuthSunFP=[]
    altitudeSunFP=[]
    azimuthSunSSW=[]
    altitudeSunSSW=[]
    azimuthSunHP=[]
    altitudeSunHP=[]
    azimuthSunWSW=[]
    altitudeSunWSW=[]
    
    if special=="all special events":
        for j in range(96):
            gatech.date =str(dateFP) +' '+str(j/4.)+":00:00"
            sun=ephem.Sun()
            sun.compute(gatech)
            azimuthSunFP.append(float(sun.az*180/3.14))
            altitudeSunFP.append(float(sun.alt*180/3.14))
            gatech.date =str(dateSSW) +' '+str(j/4.)+":00:00"
            sun=ephem.Sun()
            sun.compute(gatech)
            azimuthSunSSW.append(float(sun.az*180/3.14))
            altitudeSunSSW.append(float(sun.alt*180/3.14))
            gatech.date =str(dateHP) +' '+str(j/4.)+":00:00"
            sun=ephem.Sun()
            sun.compute(gatech)
            azimuthSunHP.append(float(sun.az*180/3.14))
            altitudeSunHP.append(float(sun.alt*180/3.14))
            gatech.date =str(dateWSW) +' '+str(j/4.)+":00:00"
            sun=ephem.Sun()
            sun.compute(gatech)
            azimuthSunWSW.append(float(sun.az*180/3.14))
            altitudeSunWSW.append(float(sun.alt*180/3.14))

    maxStarArray=[]
    if len(altitudeStar1) > 0:
        maxStarArray.append(max(altitudeStar1))
    else:
        maxStarArray.append(-100)
    if len(altitudeStar2) > 0:
        maxStarArray.append(max(altitudeStar2))
    else:
        maxStarArray.append(-100)
    if len(altitudeStar3) > 0:
        maxStarArray.append(max(altitudeStar3))
    else:
        maxStarArray.append(-100)
    
    maxStar= max(maxStarArray)
   
    if zoom==False:
        ymax = max(altitudeMoon+altitudeSun) +2
        if ymax > maxStar:
            ymax=ymax
        if ymax <= maxStar:
            ymax = maxStar
        xmin = 0
        xmax = 360
        ymin = 0
        ymax = ymax
    if zoom==True:
        xmin = xmin
        xmax = xmax
        ymin = ymin
        ymax = ymax
     
    fig, ax = plt.subplots(1, figsize=(20,9))
    plt.xlim(xmin, xmax)
    plt.ylim(ymin,ymax)
    
    #Horizontprofil
    plt.plot(profilAz,profilAlt , 'k', label="visible horizon")
    plt.plot([0,360],[0,0] , 'k--', label="math. horizon")
  
    
    #Plot Tore
    authorlist= list(anlage["gate"].keys())
    colorlist = ["blue", "green", "orange"]
    for k in range(len(authorlist)):

        for m in range(len(anlage["gate"][authorlist[k]])):
            if k== 0:
                if m == 0:
                    plt.fill_between([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][1]],[ymax, ymax], label="gate:  (acc. to "+str(authorlist[k])+ ")", facecolor=colorlist[k],alpha=0.5 )
                else:
                    plt.fill_between([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][1]],[ymax, ymax], facecolor=colorlist[k], alpha=0.5)
            if k==1:
                if m == 0:
                    plt.plot([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][0]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.plot([anlage["gate"][authorlist[k]][m][1],anlage["gate"][authorlist[k]][m][1]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.fill_between([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][1]],[ymax, ymax], label="gate:  (acc. to "+str(authorlist[k])+ ")", facecolor="none",
                                     hatch="///", edgecolor="black", linewidth=0.0 )
                else:
                    plt.plot([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][0]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.plot([anlage["gate"][authorlist[k]][m][1],anlage["gate"][authorlist[k]][m][1]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.fill_between([anlage["gate"][authorlist[k]][m][0],anlage["gate"][authorlist[k]][m][1]],[ymax, ymax],facecolor="none",
                                     hatch="///", edgecolor="black", linewidth=0.0 )


    #Plot Special Features
    authorlist= list(anlage["specialfeatures"].keys())
    colorlist = ["lightblue",  "lightgreen"]

    
    for k in range(len(authorlist)):
        for m in range(len(anlage["specialfeatures"][authorlist[k]])):
           
            if k==0:
                if m == 0:
                    plt.plot([anlage["specialfeatures"][authorlist[k]][m][0], anlage["specialfeatures"][authorlist[k]][m][0]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.plot([anlage["specialfeatures"][authorlist[k]][m][1], anlage["specialfeatures"][authorlist[k]][m][1]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.fill_between([anlage["specialfeatures"][authorlist[k]][m][0],anlage["specialfeatures"][authorlist[k]][m][1]],[ymax, ymax], label="special features: (acc. to " +str(authorlist[k])+")", facecolor="none", hatch="\\\\", edgecolor="black", linewidth=0.0)
                else:
                    plt.plot([anlage["specialfeatures"][authorlist[k]][m][0], anlage["specialfeatures"][authorlist[k]][m][0]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.plot([anlage["specialfeatures"][authorlist[k]][m][1], anlage["specialfeatures"][authorlist[k]][m][1]],[0,ymax],ls="-",c="black", linewidth=0.7)
                    plt.fill_between([anlage["specialfeatures"][authorlist[k]][m][0],anlage["specialfeatures"][authorlist[k]][m][1]],[ymax, ymax], facecolor="none",hatch="\\\\", edgecolor="black", linewidth=0.0)
            if k==1:
                if m == 0:
                    plt.fill_between([anlage["specialfeatures"][authorlist[k]][m][0],anlage["specialfeatures"][authorlist[k]][m][1]],[ymax, ymax], label="special features: (acc. to " +str(authorlist[k])+")", facecolor=colorlist[k],alpha=0.7 )
                else:
                    plt.fill_between([anlage["specialfeatures"][authorlist[k]][m][0],anlage["specialfeatures"][authorlist[k]][m][1]],[ymax, ymax], facecolor=colorlist[k], alpha=0.7)
     
    if special != "all special events":
        if starname1 != "no star":
            plt.plot(azimuthStar1,altitudeStar1, 'k', label=str(starname1)+', mag: '+str(mag1))
        if starname2 != "no star":
            plt.plot(azimuthStar2,altitudeStar2, 'b', label=str(starname2)+', mag: '+str(mag2))
        if starname3 != "no star":
            plt.plot(azimuthStar3,altitudeStar3, 'r', label=str(starname3)+', mag: '+str(mag3))
        plt.plot(azimuthMoon,altitudeMoon , 'g', label="moon")
        plt.plot(azimuthSun,altitudeSun , 'orange', label="sun")
        
    if special == "all special events":
        plt.plot(azimuthSunFP,altitudeSunFP , 'orange', label="vernal equinox")
        plt.plot(azimuthSunSSW,altitudeSunSSW , 'black', label="summer solstice")
        plt.plot(azimuthSunHP,altitudeSunHP , 'red', label="autumnal equinox")
        plt.plot(azimuthSunWSW,altitudeSunWSW , 'blue', label="winter solstice")
    
    plt.legend(bbox_to_anchor=(0.78, 0.95), loc=2, borderaxespad=0.)
    plt.xlabel('Azimuth [°]')
    plt.ylabel('Altitude [°]')
    plt.title("object: "+anlage["object"]+' (latitude: '+str(anlage["coordinates"]["latitude"])+ '°;  longitude: '+str(anlage["coordinates"]["longitude"])+  '°)\n date: '+str(date)+'\n pressure: '+str(pressureval)+' mBar '+'ecliptic: '+str(eclipt)+'°')
    return plt.show()

# Visualization

In [6]:
# WIDGET 
#Objects=["QuedlinburgI","QuedlinburgII", "Hopferstadt", "Bielen","Ippesheim","Ippesheim_2","Kyhna1", "Kyhna2", "Kyhna3", "Kyhna4", "Goseck", "Warburg-Daseburg","Bochum-Harpen", "Gneiding", "Kothingeichendorf", "Watenstedt", "Dresden-NickernI", "Dresden-NickernII",  "Bochow", "Eythra2", "Dresden3", "Kuenzing-Unternberg", "Meisternthal", "Mirskofen", "Ramsdorf", "Schmierdorf1", "Schmierdorf2", "Stephansposching", "Viecht", "Koetschlitz"]
Objects={"QuedlinburgI":"KGAL0003","QuedlinburgII":"KGAL0007", "Hopferstadt":"KGAL0023", "Bielen":"KGAL0011","Ippesheim":"KGAL0027","Goseck":"KGAL0019", 
         "Dresden-NickernI":"KGAL0039", "Dresden-NickernII":"KGAL0043",  "Bochow":"KGAL0015", "Dresden-NickernIII":"KGAL0047", "Koetschlitz":"KGAL0031"}
import warnings
warnings.filterwarnings('ignore')

starlist=sorted(starsforwidget)
starlist.insert(0,"no star")

out1 = Output()
par1a=Dropdown(
    options=sorted(Objects.keys()),
    description='object'
)
par2a=Text(
    continuous_update=False,value="-4700",description='year',layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),
    disabled=False
)
par3a=Dropdown(
    continuous_update=False,options=[1,2,3,4,5,6,7,8,9,10,11,12],description='month',layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),
    disabled=False
)
par4a=Dropdown(
    continuous_update=False,options=range(1,32),description='day',layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),
    disabled=False
)
par4b=Dropdown(
    continuous_update=False,options=['no selection', 'all special events', 'vernal_equinox','summer_solstice','autumnal_equinox','winter_solstice'],
    value="no selection",description=' ',layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),disabled=False
)
par5a= Dropdown(
    continuous_update=False,options=starlist,value="no star",description='star 1',layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),
    disabled=False
)
par5b= Dropdown(
    continuous_update=False,options=starlist,value="no star",description='star 2',layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),
    disabled=False
)

par5c= Dropdown(
    continuous_update=False,options=starlist,value="no star",description='star 3',layout=Layout(margin='10px -10px 10px -10px', min_height='40px', width='300px'),
    disabled=False
)
par6xmin2=IntSlider(
    value=0,min=0,max=360,step=1,description='az. min [°]',disabled=False,continuous_update=False,orientation='horizontal',readout=True,
    readout_format='d'
)
par6xmax2=IntSlider(
    value=360,min=0,max=360,step=1,description='az. max [°]',disabled=False,continuous_update=False,orientation='horizontal',readout=True,
    readout_format='d'
)
par6ymin2=FloatSlider(
    value=0,min=-2,max=90,step=0.2,description='alt. min [°]',disabled=False,continuous_update=False,orientation='horizontal',readout=True,
    readout_format='d'
)
par6ymax2=FloatSlider(
    value=3,min=0,max=90,step=0.2,description='alt. max [°]',disabled=False,continuous_update=False,orientation='horizontal',readout=True,
    readout_format='d'
)
parPressure=IntSlider(
    value=1013,min=0,max=2000,step=1,description='p [mBar]',disabled=False,continuous_update=False,orientation='horizontal',readout=True,
    readout_format='d'
)
par7a=Button(
   description='RUN'
)
par8a=Button(
   description='RUN with Zoom'
)
def SearchConstellation1(res):
    firstpar =par1a.value
    secondpar = str(par2a.value)
    thirdpar=str(par3a.value)
    fourthpar=str(par4a.value)
    fourthparB=str(par4b.value)
    fivthparA=par5a.value
    fivthparB=par5b.value
    fivthparC=par5c.value
    pressurevar=parPressure.value
    data=secondpar+'/'+thirdpar+'/'+fourthpar
    objectfile=Objects[par1a.value]
    
    with out1:
        print("test")
        clear_output()
        AzimuthAltitudeFull(data, str(fivthparA), str(fivthparB), str(fivthparC), objectfile, fourthparB,0, 360, 0, 90, pressurevar, zoom=False)
    return; 
    
def SearchConstellation1Zoom(res):
    firstpar =par1a.value
    secondpar = str(par2a.value)
    thirdpar=str(par3a.value)
    fourthpar=str(par4a.value)
    fourthparB=str(par4b.value)
    fivthparA=par5a.value
    fivthparB=par5b.value
    fivthparC=par5c.value
    sixthxmin2=par6xmin2.value
    sixthxmax2=par6xmax2.value
    sixthymin2=par6ymin2.value
    sixthymax2=par6ymax2.value
    pressurevar=parPressure.value
    data=secondpar+'/'+thirdpar+'/'+fourthpar
    objectfile=Objects[par1a.value]
  
    with out1:
        clear_output()
        AzimuthAltitudeFull(data,  str(fivthparA), str(fivthparB), str(fivthparC), objectfile, fourthparB, sixthxmin2, sixthxmax2, sixthymin2, sixthymax2,pressurevar, zoom=True)
    return; 
par7a.on_click(SearchConstellation1)
par8a.on_click(SearchConstellation1Zoom)
display(VBox([Label('Select object:'), par1a,HBox([par2a, par3a, par4a]), Label('Select event like solstices or equinox (specific year required)'), par4b,Label('Define pressure: default value is 1013 mBar. The value 0 mBar means airless model without refraction'), parPressure, Label('Select up to three stars'), HBox([ par5a,  par5b, par5c]),  par7a,Label('Define zoom range'), HBox([par6xmin2, par6xmax2,par6ymin2,par6ymax2]),par8a ]))
display(out1)

Output()